# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [1]:
import pyorient

In [2]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [3]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [4]:
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [5]:
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [6]:
client.command("CREATE VERTEX CONTENT {name: 'Zeus', symbol: 'thunder'}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [7]:
for vertex in client.command("SELECT FROM V"):
    print(vertex)

{'@V':{'name': 'Zeus', 'symbol': 'thunder'},'version':1,'rid':'#9:0'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [8]:
client.command("CREATE VERTEX CONTENT {name: 'Héra', symbol: 'tiara'}")
client.command("CREATE VERTEX CONTENT {name: 'Poséidon', symbol: 'trident'}")
client.command("CREATE VERTEX CONTENT {name: 'Athena', symbol: 'helmet'}")
client.command("CREATE VERTEX CONTENT {name: 'Arès', symbol: 'weapons'}")

**Q:** Display all vertices with name = `Arès`

In [9]:
for vertex in client.command("SELECT FROM V WHERE name = 'Arès'"):
    print(vertex)

{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [10]:
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Poséidon') CONTENT {kind: 'sibling'}")

**Q:** Redisplay all vertices, discuss.

In [11]:
for vertex in client.command("SELECT FROM V"):
    print(vertex)

{'@V':{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C39E8>},'version':2,'rid':'#9:0'}
{'@V':{'name': 'Héra', 'symbol': 'tiara'},'version':1,'rid':'#10:0'}
{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3C88>},'version':2,'rid':'#11:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet'},'version':1,'rid':'#12:0'}
{'@V':{'name': 'Arès', 'symbol': 'weapons'},'version':1,'rid':'#13:0'}


**Q:** Display all edges. They are contained in the class `E`

In [12]:
for edge in client.command("SELECT FROM E"):
    print(edge)

{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938C3470>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938C3AC8>},'version':1,'rid':'#21:0'}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [13]:
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Héra') CONTENT {kind: 'sibling'}")
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Arès') CONTENT {kind: 'father'}")
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Athena') CONTENT {kind: 'father'}")
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Arès') CONTENT {kind: 'mother'}")
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Héra') TO (SELECT FROM V WHERE name = 'Zeus') CONTENT {kind: 'sibling'}")
client.command("CREATE EDGE FROM (SELECT FROM V WHERE name = 'Poséidon') TO (SELECT FROM V WHERE name = 'Zeus') CONTENT {kind: 'sibling'}")

In [14]:
for edge in client.command("SELECT FROM E"):
    print(edge)

{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938C3DA0>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938C3D30>},'version':1,'rid':'#21:0'}
{'@E':{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA2E8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA240>},'version':1,'rid':'#22:0'}
{'@E':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA4A8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA550>},'version':1,'rid':'#23:0'}
{'@E':{'kind': 'father', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA710>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA7B8>},'version':1,'rid':'#24:0'}
{'@E':{'kind': 'mother', 'out': <pyorient.otypes.OrientRecordLink object at 0x00000230938CA978>, 'in': <pyorient.otypes.OrientRecordLink object at 0x00000230938CAA58>},'version':1,'rid':'#25:0'}
{'@E':{'kind': 'sibling

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [15]:
for vertex in client.command("SELECT EXPAND(out()) FROM V WHERE name = 'Zeus'"):
    print(vertex)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3CC0>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3630>},'version':3,'rid':'#11:0'}
{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3F98>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3AC8>},'version':4,'rid':'#10:0'}
{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3470>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938CA358>},'version':2,'rid':'#12:0'}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [16]:
for vertex in client.command("SELECT EXPAND(in) FROM E WHERE kind = 'father'"):
    print(vertex)

{'@V':{'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3CC0>, 'name': 'Arès', 'symbol': 'weapons'},'version':3,'rid':'#13:0'}
{'@V':{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3F98>},'version':2,'rid':'#12:0'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [17]:
for vertex in client.command("SELECT EXPAND(out) FROM E WHERE kind = 'mother'"):
    print(vertex)

{'@V':{'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3D68>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3FD0>},'version':4,'rid':'#10:0'}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [18]:
for vertex in client.command("SELECT EXPAND(in) FROM E WHERE kind = 'sibling' AND out.name = 'Zeus'"):
    print(vertex)

{'@V':{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3198>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C38D0>},'version':3,'rid':'#11:0'}
{'@V':{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938C3EF0>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x00000230938CA358>},'version':4,'rid':'#10:0'}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [19]:
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)
client.db_open("eCommerce", "root", ROOT_PASSWORD)

client.command("CREATE CLASS CUSTUMER EXTENDS V")
client.command("CREATE CLASS PRODUCT EXTENDS V")
client.command("CREATE CLASS PURCHASE EXTENDS E")

[13]

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [20]:
client.command("CREATE VERTEX PRODUCT CONTENT {name: 'spaghetti'}")
client.command("CREATE VERTEX PRODUCT CONTENT {name: 'bolognese sauce'}")
client.command("CREATE VERTEX PRODUCT CONTENT {name: 'cheese'}")
client.command("CREATE VERTEX PRODUCT CONTENT {name: 'apple'}")

for vertex in client.command("SELECT FROM PRODUCT"):
    print(vertex)

{'@PRODUCT':{'name': 'spaghetti'},'version':1,'rid':'#45:0'}
{'@PRODUCT':{'name': 'bolognese sauce'},'version':1,'rid':'#46:0'}
{'@PRODUCT':{'name': 'cheese'},'version':1,'rid':'#47:0'}
{'@PRODUCT':{'name': 'apple'},'version':1,'rid':'#48:0'}


**Q:** Create the following people: `peter`, `meredith`.

In [21]:
client.command("CREATE VERTEX CUSTUMER CONTENT {name: 'peter'}")
client.command("CREATE VERTEX CUSTUMER CONTENT {name: 'meredith'}")

for vertex in client.command("SELECT FROM CUSTUMER"):
    print(vertex)

{'@CUSTUMER':{'name': 'peter'},'version':1,'rid':'#33:0'}
{'@CUSTUMER':{'name': 'meredith'},'version':1,'rid':'#34:0'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [22]:
# - peter > spaghetti + cheese on 20/01/2016 
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'peter') TO (SELECT FROM PRODUCT WHERE name = 'spaghetti') CONTENT {date: '20/01/2016'}")

# - meredith > cheese + apple + bolognese sauce on 22/01/2016
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'meredith') TO (SELECT FROM PRODUCT WHERE name = 'cheese') CONTENT {date: '22/01/2016'}")
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'meredith') TO (SELECT FROM PRODUCT WHERE name = 'apple') CONTENT {date: '22/01/2016'}")
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'meredith') TO (SELECT FROM PRODUCT WHERE name = 'bolognese sauce') CONTENT {date: '22/01/2016'}")

# - peter > spaghetti + bolognese sauce on 27/01/2016
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'peter') TO (SELECT FROM PRODUCT WHERE name = 'spaghetti') CONTENT {date: '27/01/2016'}")
client.command("CREATE EDGE PURCHASE FROM (SELECT FROM CUSTUMER WHERE name = 'peter') TO (SELECT FROM PRODUCT WHERE name = 'bolognese sauce') CONTENT {date: '27/01/2016'}")

**Q:** Who bought Bolognese sauce?

In [35]:
for vertex in client.command("SELECT name FROM (SELECT EXPAND(out) FROM E WHERE in.name = 'bolognese sauce')"):
    print(vertex)

{{'name': 'meredith'},'version':0,'rid':'#-2:0'}
{{'name': 'peter'},'version':0,'rid':'#-2:1'}


**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [104]:
results = client.command("""
                         SELECT name AS product_name
                          FROM (
                            SELECT EXPAND(in().out())
                            FROM V
                            WHERE name = 'bolognese sauce'
                          )
                          WHERE name != 'bolognese sauce'
                        """)

# Iterate over the results and print each record
for record in results:
    print(record.oRecordData)

{'product_name': 'cheese'}
{'product_name': 'apple'}
{'product_name': 'spaghetti'}
{'product_name': 'spaghetti'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.